In [ ]:
"""This is a cookbook for fine tuning models. The example task is to improve the models ability to take 
unstructured product reviews and output them in a structured json format. 
SCROLL DOWN TO BOTTOM FOR ACTUAL FUNCTION CALLS"""
!pip uninstall -y langchain openai
!pip install langchain
!pip install openai==0.27.9

In [ ]:
import openai
openai.api_key = "<YOUR OPENAI_API_KEY>"
import os
# Set environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "<YOUR LANGSMITH API KEY>"
# the langsmith project to organize your runs under. This will happen automagically after env variables are set.
os.environ["LANGCHAIN_PROJECT"] = "product-reviews"

In [ ]:
fine_tuning_datalist = [{
    "messages": [{
        "role": "system",
        "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
    },
        {
            "role": "user",
            "content": "This coffee machine is amazing. I'd rate it 5 stars. Tastes like high-end café coffee and it's so easy to use. Takes up a bit of counter space, though."
        },
        {
            "role": "assistant",
            "content": """{"product": "coffee machine","star_rating": 5,"specific_likes": ["Tastes like high-end café coffee", "easy to use"],"specific_dislikes": ["Takes up a bit of counter space"]}"""
        }
    ]
},
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "I'm giving this air purifier a 2-star rating. While it does help a bit with allergies, it's extremely loud, especially on the higher settings."
            },
            {
                "role": "assistant",
                "content": """{"product": "air purifier","star_rating": 2,"specific_likes": ["Helps with allergies"],"specific_dislikes": ["Extremely loud on higher settings"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "The mountain bike is pretty good for the price, so I'll rate it 3 stars. It handles well on rough terrains but started to rust a bit after a few months."
            },
            {
                "role": "assistant",
                "content": """{"product": "mountain bike","star_rating": 3,"specific_likes": ["Handles well on rough terrains"],"specific_dislikes": ["Started to rust after a few months"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This stand mixer gets a 4-star rating from me. It mixes well and has a lot of different attachments. However, it's pretty bulky and takes up a lot of counter space."
            },
            {
                "role": "assistant",
                "content": """{"product": "stand mixer","star_rating": 4,"specific_likes": ["Mixes well", "has a lot of different attachments"],"specific_dislikes": ["Bulky and takes up a lot of counter space"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "I'm giving this humidifier 1 star. It leaked water all over my wooden furniture and ruined it."
            },
            {
                "role": "assistant",
                "content": """{"product": "humidifier","star_rating": 1,"specific_likes": [],"specific_dislikes": ["Leaked water and ruined wooden furniture"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This suitcase deserves a 5-star rating. It has tons of compartments and is super lightweight. The wheels glide smoothly too. Just wish it came in more colors."
            },
            {
                "role": "assistant",
                "content": """{"product": "suitcase","star_rating": 5,"specific_likes": ["Tons of compartments", "super lightweight", "wheels glide smoothly"],"specific_dislikes": ["Limited color options"]}"""
            }
        ]
    }, {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "I'm giving these noise-cancelling headphones 4 stars. Great sound quality and comfortable to wear. Battery life could be better though."
            },
            {
                "role": "assistant",
                "content": """{"product": "noise-cancelling headphones","star_rating": 4,"specific_likes": ["Great sound quality", "comfortable to wear"],"specific_dislikes": ["Battery life could be better"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This laptop is disappointing. 2 stars. It's slow and crashes frequently. The only good thing is the display."
            },
            {
                "role": "assistant",
                "content": """{"product": "laptop","star_rating": 2,"specific_likes": ["Good display"],"specific_dislikes": ["Slow", "crashes frequently"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "These running shoes are just what I needed. 5 stars. Super comfortable and great grip."
            },
            {
                "role": "assistant",
                "content": """{"product": "running shoes","star_rating": 5,"specific_likes": ["Super comfortable", "great grip"],"specific_dislikes": []}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "1 star for this hair dryer. It's loud and not very powerful. Totally not worth the money."
            },
            {
                "role": "assistant",
                "content": """{"product": "hair dryer","star_rating": 1,"specific_likes": [],"specific_dislikes": ["Loud", "not very powerful"]}"""
            }
        ]
    }, {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "The vacuum cleaner is quite effective. I would rate it 4 stars. The only downside is that it's a bit loud."
            },
            {
                "role": "assistant",
                "content": """{"product": "vacuum cleaner","star_rating": 4,"specific_likes": ["Quite effective"],"specific_dislikes": ["A bit loud"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This blender is not worth the money. One star. It leaks and the motor is weak."
            },
            {
                "role": "assistant",
                "content": """{"product": "blender","star_rating": 1,"specific_likes": [],"specific_dislikes": ["Leaks", "weak motor"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "I'll give this smartwatch 3 stars. It has many features, but the battery drains too quickly."
            },
            {
                "role": "assistant",
                "content": """{"product": "smartwatch","star_rating": 3,"specific_likes": ["Many features"],"specific_dislikes": ["Battery drains quickly"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "I'm giving this water bottle 5 stars. It keeps my drink cold for hours and is leak-proof."
            },
            {
                "role": "assistant",
                "content": """{"product": "water bottle","star_rating": 5,"specific_likes": ["Keeps drink cold for hours", "leak-proof"],"specific_dislikes": []}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This tablet gets a 2-star rating. The screen resolution is good, but it lags and freezes often."
            },
            {
                "role": "assistant",
                "content": """{"product": "tablet","star_rating": 2,"specific_likes": ["Good screen resolution"],"specific_dislikes": ["Lags and freezes often"]}"""
            }
        ]
    }, {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "I'm giving this e-reader 4 stars. It's lightweight and easy to read in the sun, but the interface could be smoother."
            },
            {
                "role": "assistant",
                "content": """{"product": "e-reader","star_rating": 4,"specific_likes": ["Lightweight", "easy to read in the sun"],"specific_dislikes": ["Interface could be smoother"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This electric toothbrush is not worth 5 stars. The battery life is short, and it's a bit noisy."
            },
            {
                "role": "assistant",
                "content": """{"product": "electric toothbrush","star_rating": 3,"specific_likes": [],"specific_dislikes": ["Short battery life", "a bit noisy"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This camping tent is amazing, 5 stars! Easy to set up and incredibly durable."
            },
            {
                "role": "assistant",
                "content": """{"product": "camping tent","star_rating": 5,"specific_likes": ["Easy to set up", "incredibly durable"],"specific_dislikes": []}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This food processor is just okay. Giving it 3 stars. It chops well, but it's hard to clean."
            },
            {
                "role": "assistant",
                "content": """{"product": "food processor","star_rating": 3,"specific_likes": ["Chops well"],"specific_dislikes": ["Hard to clean"]}"""
            }
        ]
    },
    {
        "messages": [{
            "role": "system",
            "content": "Given a product review, provide the following fields in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", and \"specific_dislikes\".The json dict must have no new lines to pretty print it, it must be on a single line."
        },
            {
                "role": "user",
                "content": "This desk chair gets 2 stars from me. It's uncomfortable and the height adjustment is faulty."
            },
            {
                "role": "assistant",
                "content": """{"product": "desk chair","star_rating": 2,"specific_likes": [],"specific_dislikes": ["Uncomfortable", "faulty height adjustment"]}"""
            }
        ]
    }
]

assistant_content_strings = [
    """{"product": "hairdryer", "star_rating": 5, "specific_likes": ["Dries hair quickly", "Not too loud"], "specific_dislikes": []}""",
    """{"product": "laptop bag", "star_rating": 3, "specific_likes": ["Fits computer"], "specific_dislikes": ["Shoulder strap broke"]}""",
    """{"product": "coffee maker", "star_rating": 4, "specific_likes": ["Makes a great brew"], "specific_dislikes": ["Pot hard to clean"]}""",
    """{"product": "sneakers", "star_rating": 5, "specific_likes": ["Super comfy", "Stylish"], "specific_dislikes": []}""",
    """{"product": "headphones", "star_rating": 2, "specific_likes": ["Okay sound quality"], "specific_dislikes": ["Don't fit well"]}""",
    """{"product": "yoga mat", "star_rating": 5, "specific_likes": ["Non-slip", "Perfect thickness"], "specific_dislikes": []}""",
    """{"product": "wall clock", "star_rating": 3, "specific_likes": ["Looks good"], "specific_dislikes": ["Ticks too loudly"]}""",
    """{"product": "refrigerator", "star_rating": 4, "specific_likes": ["Spacious", "Cools well"], "specific_dislikes": ["Ice maker jams"]}""",
    """{"product": "hand mixer", "star_rating": 2, "specific_likes": ["It works"], "specific_dislikes": ["Beaters hard to attach"]}""",
    """{"product": "board game", "star_rating": 5, "specific_likes": ["Great for family game night"], "specific_dislikes": []}""",
    """{"product": "camera", "star_rating": 4, "specific_likes": ["Takes sharp photos"], "specific_dislikes": ["Clunky user interface"]}""",
    """{"product": "dog leash", "star_rating": 3, "specific_likes": ["Adjustable length"], "specific_dislikes": ["Material feels cheap"]}""",
    """{"product": "moisturizer", "star_rating": 5, "specific_likes": ["Skin feels soft"], "specific_dislikes": []}""",
    """{"product": "flashlight", "star_rating": 3, "specific_likes": ["Bright"], "specific_dislikes": ["Poor battery life"]}""",
    """{"product": "microwave", "star_rating": 4, "specific_likes": ["Heats food evenly"], "specific_dislikes": ["Buttons hard to press"]}""",
    """{"product": "novel", "star_rating": 5, "specific_likes": ["Couldn't put it down"], "specific_dislikes": []}""",
    """{"product": "bicycle", "star_rating": 2, "specific_likes": ["Rides okay"], "specific_dislikes": ["Chain broke"]}""",
    """{"product": "perfume", "star_rating": 5, "specific_likes": ["Scent lasts all day"], "specific_dislikes": []}""",
    """{"product": "office chair", "star_rating": 3, "specific_likes": ["Comfortable"], "specific_dislikes": ["Wheels don't move smoothly"]}""",
    """{"product": "toaster", "star_rating": 4, "specific_likes": ["Toasts bread evenly"], "specific_dislikes": ["A bit slow"]}""",
    """{"product": "vacuum cleaner", "star_rating": 5, "specific_likes": ["Picks up everything", "Easy to maneuver"], "specific_dislikes": []}""",
    """{"product": "grill", "star_rating": 3, "specific_likes": ["Cooks well"], "specific_dislikes": ["Difficult to assemble"]}""",
    """{"product": "blender", "star_rating": 4, "specific_likes": ["Powerful"], "specific_dislikes": ["A little noisy"]}""",
    """{"product": "hiking boots", "star_rating": 5, "specific_likes": ["Comfortable", "Waterproof"], "specific_dislikes": []}""",
    """{"product": "pillow", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Too firm"]}""",
    """{"product": "smartwatch", "star_rating": 5, "specific_likes": ["Accurate tracking", "Good battery life"], "specific_dislikes": []}""",
    """{"product": "luggage set", "star_rating": 3, "specific_likes": ["Spacious"], "specific_dislikes": ["Zippers get stuck"]}""",
    """{"product": "baby stroller", "star_rating": 4, "specific_likes": ["Easy to fold"], "specific_dislikes": ["A bit heavy"]}""",
    """{"product": "e-reader", "star_rating": 5, "specific_likes": ["Lightweight", "Easy on the eyes"], "specific_dislikes": []}""",
    """{"product": "water bottle", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Leaks when tipped over"]}""",
    """{"product": "treadmill", "star_rating": 4, "specific_likes": ["Easy to use"], "specific_dislikes": ["Takes up a lot of space"]}""",
    """{"product": "winter coat", "star_rating": 5, "specific_likes": ["Keeps warm in extreme cold"], "specific_dislikes": []}""",
    """{"product": "portable speaker", "star_rating": 3, "specific_likes": ["Good sound"], "specific_dislikes": ["Battery drains quickly"]}""",
    """{"product": "kitchen knife", "star_rating": 4, "specific_likes": ["Sharp"], "specific_dislikes": ["Uncomfortable handle"]}""",
    """{"product": "sunscreen", "star_rating": 5, "specific_likes": ["Non-greasy", "Long-lasting"], "specific_dislikes": []}""",
    """{"product": "curling iron", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Gets too hot", "Burns hair"]}""",
    """{"product": "electric toothbrush", "star_rating": 4, "specific_likes": ["Cleans well"], "specific_dislikes": ["A bit pricey"]}""",
    """{"product": "laptop stand", "star_rating": 3, "specific_likes": ["Sturdy"], "specific_dislikes": ["Wobbles a little"]}""",
    """{"product": "air purifier", "star_rating": 5, "specific_likes": ["Helps with allergies", "Quiet"], "specific_dislikes": []}""",
    """{"product": "food processor", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Difficult to clean"]}""",
    """{"product": "exercise bike", "star_rating": 4, "specific_likes": ["Good workout"], "specific_dislikes": ["Hard seat"]}""",
    """{"product": "frying pan", "star_rating": 5, "specific_likes": ["Non-stick", "Easy to clean"], "specific_dislikes": []}""",
    """{"product": "phone case", "star_rating": 3, "specific_likes": ["Looks good"], "specific_dislikes": ["Not very protective"]}""",
    """{"product": "umbrella", "star_rating": 4, "specific_likes": ["Durable"], "specific_dislikes": ["Hard to open"]}""",
    """{"product": "TV", "star_rating": 5, "specific_likes": ["Great picture quality", "Good sound"], "specific_dislikes": []}""",
    """{"product": "keyboard", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Keys stick", "Not ergonomic"]}""",
    """{"product": "rice cooker", "star_rating": 4, "specific_likes": ["Cooks evenly"], "specific_dislikes": ["A bit slow"]}""",
    """{"product": "printer", "star_rating": 3, "specific_likes": ["Prints well"], "specific_dislikes": ["Expensive ink"]}""",
    """{"product": "suitcase", "star_rating": 5, "specific_likes": ["Lightweight", "Rolls smoothly"], "specific_dislikes": []}""",
    """{"product": "dish rack", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Rusts easily"]}""",
    """{"product": "humidifier", "star_rating": 4, "specific_likes": ["Helps with dry air"], "specific_dislikes": ["A bit noisy"]}""",
    """{"product": "tea kettle", "star_rating": 5, "specific_likes": ["Heats up quickly", "Easy to pour"], "specific_dislikes": []}""",
    """{"product": "backpack", "star_rating": 3, "specific_likes": ["Good size"], "specific_dislikes": ["Uncomfortable straps"]}""",
    """{"product": "lawn mower", "star_rating": 4, "specific_likes": ["Cuts well"], "specific_dislikes": ["Hard to start"]}""",
    """{"product": "facial cleanser", "star_rating": 5, "specific_likes": ["Leaves skin feeling fresh"], "specific_dislikes": []}""",
    """{"product": "tablet", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Slow", "Poor battery life"]}""",
    """{"product": "fan", "star_rating": 4, "specific_likes": ["Cools well"], "specific_dislikes": ["A little bulky"]}""",
    """{"product": "wristwatch", "star_rating": 3, "specific_likes": ["Looks nice"], "specific_dislikes": ["Loses time"]}""",
    """{"product": "frying pan", "star_rating": 5, "specific_likes": ["Easy to handle", "Heats evenly"], "specific_dislikes": []}""",
    """{"product": "hairbrush", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Bristles fall out"]}""",
    """{"product": "electric kettle", "star_rating": 4, "specific_likes": ["Boils fast"], "specific_dislikes": ["Hard to open lid"]}""",
    """{"product": "notebook", "star_rating": 3, "specific_likes": ["Good paper quality"], "specific_dislikes": ["Cover comes off easily"]}""",
    """{"product": "bread maker", "star_rating": 5, "specific_likes": ["Easy to use", "Delicious results"], "specific_dislikes": []}""",
    """{"product": "bluetooth headphones", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Connection issues", "Poor sound quality"]}""",
    """{"product": "space heater", "star_rating": 4, "specific_likes": ["Heats quickly"], "specific_dislikes": ["Loud"]}""",
    """{"product": "coffee grinder", "star_rating": 3, "specific_likes": ["Compact"], "specific_dislikes": ["Inconsistent grind"]}""",
    """{"product": "sunglasses", "star_rating": 5, "specific_likes": ["Stylish", "UV protection"], "specific_dislikes": []}""",
    """{"product": "smartphone", "star_rating": 4, "specific_likes": ["Fast performance"], "specific_dislikes": ["Short battery life"]}""",
    """{"product": "dog bed", "star_rating": 3, "specific_likes": ["Soft"], "specific_dislikes": ["Not durable"]}""",
    """{"product": "speaker", "star_rating": 5, "specific_likes": ["Great sound quality", "Easy to set up"], "specific_dislikes": []}""",
    """{"product": "TV remote", "star_rating": 2, "specific_likes": [], "specific_dislikes": ["Buttons stick", "Unresponsive"]}""",
]

user_content_strings = [
    "This hairdryer is fantastic! Definitely 5 stars. Dries my hair quickly and isn't too loud.",
    "The laptop bag gets a 3 out of 5. It fits my computer, but the shoulder strap broke after a week.",
    "I'll give the coffee maker 4 stars. Makes a great brew, but the pot is a little hard to clean.",
    "These sneakers are super comfy. 5 stars for sure! They are stylish too.",
    "The headphones get 2 stars from me. Sound quality is okay, but they don't fit well over my ears.",
    "This yoga mat is excellent, worth 5 stars. It's non-slip and the perfect thickness.",
    "Giving the wall clock 3 stars. Looks good, but it ticks too loudly for my liking.",
    "This refrigerator is a solid 4 stars. Spacious and cools well, but the ice maker jams sometimes.",
    "2 stars for this hand mixer. It works, but the beaters are a pain to attach.",
    "Love this board game! It's a 5-star game for family game night.",
    "The camera gets 4 stars. Takes sharp photos, but the user interface is a bit clunky.",
    "I rate this dog leash 3 stars. The length is adjustable, but the material feels cheap.",
    "This moisturizer is amazing! 5 stars! My skin feels so soft after using it.",
    "The flashlight is okay, giving it 3 stars. It's bright but the battery life isn't great.",
    "4 stars for this microwave. Heats food evenly, but the buttons are hard to press.",
    "This novel is a masterpiece. 5 stars without a doubt! Couldn't put it down.",
    "The bicycle gets 2 stars. Rides okay, but the chain broke on my second ride.",
    "Love this perfume! It's a definite 5 stars. The scent lasts all day.",
    "This office chair gets 3 stars. Comfortable, but the wheels don't move smoothly.",
    "4 stars for the toaster. Toasts bread evenly, but it's a bit slow.",
    "This vacuum cleaner is a 5-star product! Picks up everything and easy to maneuver.",
    "This grill is just okay. Giving it 3 stars. It cooks well, but it's difficult to assemble.",
    "I'll give this blender 4 stars. It's powerful but a little noisy.",
    "These hiking boots are amazing. 5 stars for sure! Comfortable and waterproof.",
    "This pillow gets 2 stars from me. It's too firm.",
    "This smartwatch is excellent, 5 stars! Accurate tracking and good battery life.",
    "Giving this luggage set 3 stars. It's spacious but the zippers get stuck.",
    "This baby stroller is a solid 4 stars. Easy to fold but a bit heavy.",
    "I love this e-reader! 5 stars! It's lightweight and easy on the eyes.",
    "2 stars for this water bottle. It leaks when tipped over.",
    "I rate this treadmill 4 stars. Easy to use but takes up a lot of space.",
    "5 stars for this winter coat! Keeps me warm in extreme cold.",
    "This portable speaker gets 3 stars. Good sound but the battery drains quickly.",
    "4 stars for this kitchen knife. It's sharp but has an uncomfortable handle.",
    "This sunscreen is amazing! 5 stars! Non-greasy and long-lasting.",
    "2 stars for this curling iron. It gets too hot and burns my hair.",
    "4 stars for this electric toothbrush. Cleans well but a bit pricey.",
    "This laptop stand gets 3 stars. It's sturdy but wobbles a little.",
    "5 stars for this air purifier! Helps with allergies and is quiet.",
    "Giving this food processor 2 stars. It's difficult to clean.",
    "This exercise bike is 4 stars. Good workout but has a hard seat.",
    "I love this frying pan! 5 stars! Non-stick and easy to clean.",
    "I rate this phone case 3 stars. Looks good but it's not very protective.",
    "4 stars for this umbrella. It's durable but hard to open.",
    "This TV is a 5-star product! Great picture quality and good sound.",
    "2 stars for this keyboard. The keys stick and it's not ergonomic.",
    "I'll give this rice cooker 4 stars. Cooks evenly but is a bit slow.",
    "3 stars for this printer. Prints well but the ink is expensive.",
    "This suitcase is excellent, 5 stars! Lightweight and rolls smoothly.",
    "2 stars for this dish rack. It rusts easily.",
    "I rate this humidifier 4 stars. Helps with dry air but is a bit noisy.",
    "5 stars for this tea kettle! Heats up quickly and is easy to pour.",
    "Giving this backpack 3 stars. Good size but uncomfortable straps.",
    "4 stars for this lawn mower. Cuts well but is hard to start.",
    "This facial cleanser is amazing! 5 stars! Leaves skin feeling fresh.",
    "2 stars for this tablet. It's slow and has poor battery life.",
    "I'll give this fan 4 stars. Cools well but is a little bulky.",
    "I rate this wristwatch 3 stars. Looks nice but loses time.",
    "5 stars for this frying pan! Easy to handle and heats evenly.",
    "2 stars for this hairbrush. The bristles fall out.",
    "4 stars for this electric kettle. Boils fast but hard to open the lid.",
    "This notebook is okay, giving it 3 stars. Good paper quality but the cover comes off easily.",
    "Love this bread maker! 5 stars! Easy to use and delivers delicious results.",
    "2 stars for these Bluetooth headphones. Connection issues and poor sound quality.",
    "I rate this space heater 4 stars. Heats quickly but is loud.",
    "3 stars for this coffee grinder. Compact but has an inconsistent grind.",
    "5 stars for these sunglasses! Stylish and offer UV protection.",
    "This smartphone is a solid 4 stars. Fast performance but short battery life.",
    "I rate this dog bed 3 stars. Soft but not durable.",
    "5 stars for this speaker! Great sound quality and easy to set up.",
    "Giving this TV remote 2 stars. Buttons stick and it's unresponsive."
]


In [ ]:
import asyncio
import json
import os
from time import sleep
from langsmith import Client

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.smith import run_on_dataset, RunEvalConfig, arun_on_dataset


def write_to_jsonl():
    """writes the fine_tuning_datalist to jsonl, one json object per line. This is the format required by openai for finetuning."""
    with open('mydata.jsonl', 'w') as f:
        for entry in fine_tuning_datalist:
            json_str = json.dumps(entry)
            f.write(json_str + '\n')


def fine_tune_model():
    """fine tunes the model with the fine_tuning_datalist"""
    write_to_jsonl()
    file_response = openai.File.create(
        file=open("mydata.jsonl", "rb"),
        purpose='fine-tune'
    )
    sleep(25)
    job_response = openai.FineTuningJob.create(
        training_file=file_response["id"],
        model="gpt-3.5-turbo",
        hyperparameters={"n_epochs": 10} # the number of times to run the training data through the model. 

    )
    return job_response


def print_fine_tune_jobs():
    """shows jobs ordered in ascending order by creation timestamp"""
    print(openai.FineTuningJob.list(limit=10))

    
async def model_completion(prompts, model):
    """Runs the model which will add the runs to your project in langchain"""
    tasks = [
        ChatOpenAI(model=model).agenerate([[SystemMessage(content="""Given a product review, provide the 
                following fields in a JSON dict, where applicable: \"product\", \"star_rating\", 
                \"specific_likes\", and \"specific_dislikes\"."""),
                                            HumanMessage(content=prompt)] for prompt in prompts])
    ]
    results = await asyncio.gather(*tasks)
    print(f"\n\n{model.upper()}\n\n")
    for i, completion in enumerate(results[0].generations):  # Assuming results[0] contains the list of model outputs
        print(f"model response for '{prompts[i]}': {completion[0].text}")


async def run_models_on_prompt(model="gpt-3.5-turbo-0613", ):
    """Creates tagged runs for each model in your langsmith project. 
    You can later add those runs to a dataset for testing, or evaluate latency and other metrics."""
    
    prompts = [
        "This hairdryer is fantastic! Definitely 5 stars. Dries my hair quickly and isn't too loud.",
        "The laptop bag gets a 3 out of 5. It fits my computer, but the shoulder strap broke after a week.",
        "I'll give the coffee maker 4 stars. Makes a great brew, but the pot is a little hard to clean.",
        "These sneakers are super comfy. 5 stars for sure! They are stylish too.",
        "The headphones get 2 stars from me. Sound quality is okay, but they don't fit well over my ears.",
        "This yoga mat is excellent, worth 5 stars. It's non-slip and the perfect thickness.",
        "Giving the wall clock 3 stars. Looks good, but it ticks too loudly for my liking.",
        "This refrigerator is a solid 4 stars. Spacious and cools well, but the ice maker jams sometimes.",
        "2 stars for this hand mixer. It works, but the beaters are a pain to attach.",
        "Love this board game! It's a 5-star game for family game night.",
        "The camera gets 4 stars. Takes sharp photos, but the user interface is a bit clunky.",
        "I rate this dog leash 3 stars. The length is adjustable, but the material feels cheap.",
        "This moisturizer is amazing! 5 stars! My skin feels so soft after using it.",
        "The flashlight is okay, giving it 3 stars. It's bright but the battery life isn't great.",
        "4 stars for this microwave. Heats food evenly, but the buttons are hard to press.",
        "This novel is a masterpiece. 5 stars without a doubt! Couldn't put it down.",
        "The bicycle gets 2 stars. Rides okay, but the chain broke on my second ride.",
        "Love this perfume! It's a definite 5 stars. The scent lasts all day.",
        "This office chair gets 3 stars. Comfortable, but the wheels don't move smoothly.",
        "4 stars for the toaster. Toasts bread evenly, but it's a bit slow."
    ]

    await asyncio.gather(model_completion(prompts, model=model))
    await asyncio.sleep(5)


def create_dataset(dataset_name=None):
    """adds an example run with inputs and outputs to an existing dataset"""
    client = Client()
    
    dataset_name = dataset_name
    dataset = client.create_dataset(dataset_name=dataset_name)
    return dataset_name

def add_to_dataset(dataset_name):
    client = Client()

    dataset = client.read_dataset(dataset_name=dataset_name)
    for idx, example in enumerate(user_content_strings):
        client.create_chat_example(
            messages=[{"type": "system", "data": {"content": f"""Given a product review, provide the following fields 
            in a JSON dict, where applicable: \"product\", \"star_rating\", \"specific_likes\", 
            and \"specific_dislikes\". The json dict must have no new lines to pretty print it, it must be on a 
            single line."""}},
                      {"type": "human", "data": {"content": f"{example}"}}],
            generations={"type": "ai", "data": {"content": f"""{assistant_content_strings[idx]}"""}},
            dataset_id=dataset.id,
        )




async def evaluate_dataset(dataset_name=None, num_repetitions=3, model="gpt-3.5-turbo-0613", project_name=None):
    """runs the model you want to evaluate against the assumed to be correct examples in your dataset, grading the evaluated model output correct or incorrect."""
    
    from langchain.smith import run_on_dataset, RunEvalConfig, arun_on_dataset
    # The chat model you want to test
    model_to_test = ChatOpenAI(temperature=0, model=model)

    client = Client()
    
    """runs a question/answer evaluation, where the eval llm (gpt-4) will determine
    if model_to_test's outputs are correct based on the example_dataset we uploaded in the previous set.
    the example_dataset is treated by the eval as a correct answer for the given input."""
    
    eval_config = RunEvalConfig(
        evaluators=[
            "cot_qa"
        ],
    )
    chain_results = await arun_on_dataset(
        client,
        dataset_name=dataset_name,
        llm_or_chain_factory=model_to_test,
        evaluation=eval_config,
        num_repetitions=num_repetitions,
        project_name=project_name
    )


def get_feedback_stats(dataset_name=None):
    """Gets the evaluation overview stats for feedback run on the given dataset."""
    client = Client()

    runs = list(client.list_runs(dataset_name=dataset_name,limit=1))
    for run in runs:
        print(client.read_project(project_id=run.session_id).feedback_stats)




In [ ]:
EXAMPLE_DATASET_NAME = "<YOUR NAME HERE>"

In [ ]:
"""Fine tune your openai models here. This can take anywhere from 10 minutes to several hours
depending on openai's current load."""
fine_tune_model()
print_fine_tune_jobs()

In [ ]:
"""will tell you the status of the fine tuning for each model along with the identifer string if succeeded. You'll also receive an email from openai
when the job has succeeded with the model identifier string. The format of the identifier string is:
ft:gpt-3.5-turbo-0613:<ORG_NAME>::<MODEL_ID>"""
print_fine_tune_jobs()

In [ ]:
"""Run this to create your dataset, then add the example_data to your new dataset so we can use it in an evaluation."""
from langsmith import Client




try:
    create_dataset(EXAMPLE_DATASET_NAME)
except ValueError:
    pass # the dataset name already exists
add_to_dataset(dataset_name=EXAMPLE_DATASET_NAME)

In [ ]:
"""Evaluate your dataset using the baseline gpt-3.5-turbo."""
%autoawait

# run the baseline model
await evaluate_dataset(dataset_name=EXAMPLE_DATASET_NAME,  project_name="baseline-gpt-3.5-turbo")
# prevents rate limiting by openai and allows the eval to complete
await asyncio.sleep(10)



In [ ]:
# For the baseline gpt-3.5-turbo get the accuracy states measured as number of correct runs/total runs
get_feedback_stats(dataset_name=EXAMPLE_DATASET_NAME)

In [ ]:
"""And now evaluate your dataset using your finetuned model."""
# should be in format of ft:gpt-3.5-turbo-0613:<ORG_NAME>::<MODEL_ID>
FINE_TUNED_MODEL = "ft:gpt-3.5-turbo-0613:asimov-ai::7rUaA08g"
# run your fine tuned model
%autoawait
await evaluate_dataset(dataset_name=EXAMPLE_DATASET_NAME, model=FINE_TUNED_MODEL, project_name="finetuned-gpt-3.5-turbo")
# prevents rate limiting by openai and allows the eval to complete
await asyncio.sleep(10)

In [ ]:
# For your fine tuned model get the accuracy states measured as number of correct runs/total runs
get_feedback_stats(dataset_name=EXAMPLE_DATASET_NAME)

In [ ]:
"""Run this to print out the output of your different models. The runs will be automatically tagged 
with the model name and added to the project 'product-reviews'."""
%autoawait
await run_models_on_prompt()